In [ ]:
import pandas as pd
import walnut

# Example 5

### More complex convolutional neural network

The goal of this model is to classify images of clothing items.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
data = pd.read_csv('data/fashion_mnist/fashion-mnist_train.csv')
data.head()

In [ ]:
data_enc = walnut.preprocessing.encoding.pd_one_hot_encode(data, columns=['label'])
data_enc.head()

In [ ]:
tensor = walnut.pd_to_tensor(data_enc)
train, val, test = walnut.preprocessing.split_train_val_test(tensor, ratio_val=0.005, ratio_test=0.01)

In [ ]:
x_train, y_train = walnut.preprocessing.split_features_labels(train, 784)
x_val, y_val = walnut.preprocessing.split_features_labels(val, 784)
x_test, y_test = walnut.preprocessing.split_features_labels(test, 784)

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 1 , 28, -1))
x_val = x_val.reshape((x_val.shape[0], 1, 28, -1))
x_test = x_test.reshape((x_test.shape[0], 1, 28, -1))

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')

print (f'{x_val.shape=}')
print (f'{y_val.shape=}')

print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

Normalization

In [ ]:
x_train = x_train / 255
x_val = x_val / 255
x_test = x_test / 255

### Step 2: Build the neural network structure

In [ ]:
import walnut.nn as nn

model = nn.Sequential(layers=[
    nn.layers.Convolution2d(32, input_shape=(1, 28, 28), kernel_size=(3, 3), act="relu", norm="batch"),
    nn.layers.MaxPooling(p_window=(2, 2)),
    nn.layers.Reshape(),
    nn.layers.Linear(64, act="relu", norm="batch"),
    nn.layers.Linear(10, act="softmax", norm="batch")
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(l_r=1e-3),
    loss_fn=nn.losses.Crossentropy(),
    metric=nn.metrics.Accuracy()
)

In [ ]:
model

### Step 3: Train the model

In [ ]:
train_loss_hist, val_loss_hist = model.train(x_train, y_train, epochs=100, batch_size=512, val_data=(x_val, y_val))

In [ ]:
n = 10
traces = {
    "train_loss" : [l if i < n else sum(train_loss_hist[i-(n-1):i+1])/n for i,l in enumerate(train_loss_hist)],
    "val_loss" : val_loss_hist
}
nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="loss history", x_label="epoch", y_label="loss")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')

In [ ]:
predictions = model(x_test)
nn.analysis.plot_confusion_matrix(predictions, y_test, figsize=(5, 5), cmap='Blues')